In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd '/content/gdrive/MyDrive/wj-cnn'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/wj-cnn


In [ ]:
import random
import pickle
import matplotlib.pyplot as plt
import numpy as np
from torchvision import transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import time
if_use_gpu=torch.cuda.is_available()
print("GPU:",if_use_gpu)
def seed_torch(seed=1029):
	random.seed(seed)
	np.random.seed(seed)
	torch.manual_seed(seed)  # CPU
	torch.cuda.manual_seed(seed)  # GPU
	torch.cuda.manual_seed_all(seed)  # Multi-GPU.
	torch.backends.cudnn.benchmark = False
	torch.backends.cudnn.deterministic = True

seed=1029
seed_torch(seed)
torch.cuda.empty_cache()

GPU: True


In [ ]:
img_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

In [ ]:
class MyDataset(Dataset):
    def __init__(self, img_file, label_file, transform=None, idx = None):
        self.data = pickle.load( open( img_file, 'rb' ), encoding='bytes')
        self.targets = np.genfromtxt(label_file, delimiter=',', skip_header=1)[:,1:]
        if idx is not None:
          self.targets = self.targets[idx]
          self.data = self.data[idx]
        self.transform = transform

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, index):
        img, target = self.data[index], int(self.targets[index]) - 5  
        img = Image.fromarray(img.astype('uint8'), mode='L')
        
        if self.transform is not None:
           img = self.transform(img)

        return img, target

In [ ]:
def data_loader(batchsize):
  dataset = MyDataset('./Train.pkl', './TrainLabels.csv',transform=img_transform, idx=None)
  train_dataset, test_dataset = train_test_split(dataset, test_size=0.1)

  batch_size = batchsize  
  def _init_fn(worker_id):
      np.random.seed(int(seed)+worker_id)
  train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=16, pin_memory=True, worker_init_fn=_init_fn)  
  test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=16, pin_memory=True, worker_init_fn=_init_fn)  

  return train_loader, test_loader

In [ ]:
## Six-layer network

class Net(nn.Module):
    # This part defines the layers
    def __init__(self):
        super(Net, self).__init__()  # 1*128*128

        self.conv1 = nn.Conv2d(1, 16, kernel_size=5, padding = 2) 
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=5, padding = 2)
        self.bn2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding = 1)
        self.bn3 = nn.BatchNorm2d(64)
        
        self.fc1 = nn.Linear(16384, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 9)


    def forward(self, x):
        
        x = F.relu(F.max_pool2d(self.bn1(self.conv1(x)), 2))  
        x = F.relu(F.max_pool2d(self.bn2(self.conv2(x)), 2))  
        x = F.relu(F.max_pool2d(self.bn3(self.conv3(x)), 2))  

        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return F.log_softmax(x)

net = Net()

In [ ]:
## LeNet-5

class LeNet(nn.Module):
    # This part defines the layers
    def __init__(self):
        super(LeNet, self).__init__()  

        self.conv1 = nn.Conv2d(1, 6, kernel_size=5)
        self.bn1 = nn.BatchNorm2d(6)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.bn2 = nn.BatchNorm2d(16)

        self.fc1 = nn.Linear(13456, 120, bias = True)
        self.fc2 = nn.Linear(120, 84, bias = True)
        self.fc3 = nn.Linear(84, 9, bias = True)  


    def forward(self, x):
        
        x = F.relu(F.max_pool2d(self.bn1(self.conv1(x)), 2))  
        x = F.relu(F.max_pool2d(self.bn2(self.conv2(x)), 2))  
 
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return F.log_softmax(x)

LeNet_5 = LeNet()

In [ ]:
## AlexNet

class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 96, 11, 4), # in_channels, out_channels, kernel_size, stride, padding
            nn.BatchNorm2d(96),
            nn.ReLU(),
            nn.MaxPool2d(3, 2), # kernel_size, stride
            nn.Conv2d(96, 256, 5, 1, 2),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(3, 2),
            nn.Conv2d(256, 384, 3, 1, 1),
            nn.BatchNorm2d(384),
            nn.ReLU(),
            nn.Conv2d(384, 384, 3, 1, 1),
            nn.BatchNorm2d(384),
            nn.ReLU(),
            nn.Conv2d(384, 256, 3, 1, 1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(3, 2)
        )

        self.fc = nn.Sequential(
            nn.Linear(1024, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, 9)
        )

    def forward(self, img):
        feature = self.conv(img)
        output = self.fc(feature.view(img.shape[0], -1))
        return F.log_softmax(output)


AlexNet_8 = AlexNet()

In [ ]:
# VGG-16

class VGG1(nn.Module): 
    def __init__(self):
        super(VGG1, self).__init__()
        self.conv = nn.Sequential(
            # Block 1
            nn.Conv2d(1, 64, 3, 1, 1), # in_channels, out_channels, kernel_size, stride, padding
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, 1, 1), # in_channels, out_channels, kernel_size, stride, padding
            nn.BatchNorm2d(64),
            nn.ReLU(),            
            nn.MaxPool2d(2, 2), # kernel_size, stride

            # Block 2
            nn.Conv2d(64, 128, 3, 1, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 128, 3, 1, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(),            
            nn.MaxPool2d(2, 2),

            # Block 3
            nn.Conv2d(128, 256, 3, 1, 1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, 256, 3, 1, 1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, 256, 3, 1, 1),
            nn.BatchNorm2d(256),
            nn.ReLU(),            
            nn.MaxPool2d(2, 2),

            # Block 4
            nn.Conv2d(256, 512, 3, 1, 1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Conv2d(512, 512, 3, 1, 1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Conv2d(512, 512, 3, 1, 1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            # Block 5
            nn.Conv2d(512, 512, 3, 1, 1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Conv2d(512, 512, 3, 1, 1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Conv2d(512, 512, 3, 1, 1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )

        self.fc = nn.Sequential(
            nn.Linear(8192, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),  
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, 9),
        )

    def forward(self, img):
        feature = self.conv(img)
        output = self.fc(feature.view(img.shape[0], -1))
        return F.log_softmax(output)

VGG_16 = VGG1()

In [ ]:
# VGG-19

class VGG2(nn.Module): 
    def __init__(self):
        super(VGG2, self).__init__()
        self.conv = nn.Sequential(
            # Block 1
            nn.Conv2d(1, 64, 3, 1, 1), # in_channels, out_channels, kernel_size, stride, padding
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, 1, 1), # in_channels, out_channels, kernel_size, stride, padding
            nn.BatchNorm2d(64),
            nn.ReLU(),            
            nn.MaxPool2d(2, 2), # kernel_size, stride

            # Block 2
            nn.Conv2d(64, 128, 3, 1, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 128, 3, 1, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(),            
            nn.MaxPool2d(2, 2),

            # Block 3
            nn.Conv2d(128, 256, 3, 1, 1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, 256, 3, 1, 1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.Conv2d(256, 256, 3, 1, 1),
            nn.BatchNorm2d(256),
            nn.ReLU(),    
            nn.Conv2d(256, 256, 3, 1, 1),
            nn.BatchNorm2d(256),
            nn.ReLU(),           
            nn.MaxPool2d(2, 2),

            # Block 4
            nn.Conv2d(256, 512, 3, 1, 1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Conv2d(512, 512, 3, 1, 1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Conv2d(512, 512, 3, 1, 1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Conv2d(512, 512, 3, 1, 1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),

            # Block 5
            nn.Conv2d(512, 512, 3, 1, 1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Conv2d(512, 512, 3, 1, 1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Conv2d(512, 512, 3, 1, 1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.Conv2d(512, 512, 3, 1, 1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )

        self.fc = nn.Sequential(
            nn.Linear(8192, 4096),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),  
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(4096, 9),
        )

    def forward(self, img):
        feature = self.conv(img)
        output = self.fc(feature.view(img.shape[0], -1))
        return F.log_softmax(output)

VGG_19 = VGG2()

In [ ]:
## NiNet

class GlobalAvgPool2d(nn.Module):
    def __init__(self):
        super(GlobalAvgPool2d, self).__init__()
    def forward(self, x):
        return F.avg_pool2d(x, kernel_size=x.size()[2:])

class NiNet(nn.Module):
    def __init__(self):
        super(NiNet, self).__init__()
        self.conv = nn.Sequential(
                # NIN_1
                nn.Conv2d(1, 96, 11, 4),
                nn.BatchNorm2d(96),
                nn.ReLU(),
                nn.Conv2d(96, 96, 1),
                nn.BatchNorm2d(96),
                nn.ReLU(),
                nn.Conv2d(96, 96, 1),
                nn.BatchNorm2d(96),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=3, stride=2),

                # NIN_2
                nn.Conv2d(96, 256, 5, 1, 2),
                nn.BatchNorm2d(256),
                nn.ReLU(),
                nn.Conv2d(256, 256, 1),
                nn.BatchNorm2d(256),
                nn.ReLU(),
                nn.Conv2d(256, 256, 1),
                nn.BatchNorm2d(256),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size=3, stride=2),

                # NIN_3
                nn.Conv2d(256, 384, 3, 1, 1),
                nn.BatchNorm2d(384),
                nn.ReLU(),
                nn.Conv2d(384, 384, 1),
                nn.BatchNorm2d(384),
                nn.ReLU(),
                nn.Conv2d(384, 384, 1),
                nn.BatchNorm2d(384),
                nn.ReLU(),
                nn.Dropout(0.5),

                # NIN_4
                nn.Conv2d(384, 9, 3, 1, 1),
                nn.BatchNorm2d(9),
                nn.ReLU(),
                nn.Conv2d(9, 9, 1),
                nn.BatchNorm2d(9),
                nn.ReLU(),
                nn.Conv2d(9, 9, 1),
                nn.BatchNorm2d(9),
                nn.ReLU(),    

                GlobalAvgPool2d() 
        )

    def forward(self, img):
        feature = self.conv(img)
        output = feature.view(img.shape[0], -1)
        return F.log_softmax(output)

NiN = NiNet()

In [ ]:
## ResNet-18

class ResNet1(nn.Module):
    def __init__(self):

        super(ResNet1, self).__init__()

        # Block 1  
        self.conv1 = nn.Conv2d(1, 64, 7, 2, 3) # in_channels, out_channels, kernel_size, stride, padding  
        self.bn1 = nn.BatchNorm2d(64)

        # Block 2 Resnet_1
        self.conv2_1 = nn.Conv2d(64, 64, 3, 1, 1)
        self.bn2_1 = nn.BatchNorm2d(64)
        self.conv2_2 = nn.Conv2d(64, 64, 3, 1, 1)
        self.bn2_2 = nn.BatchNorm2d(64)   

        # Block 3 Resnet_2
        self.conv3_1 = nn.Conv2d(64, 128, 3, 1, 1)
        self.bn3_1 = nn.BatchNorm2d(128)
        self.conv3_2 = nn.Conv2d(128, 128, 3, 1, 1)
        self.bn3_2 = nn.BatchNorm2d(128)
        self.conv3_3 = nn.Conv2d(64, 128, 1, 1)
        self.bn3_3 = nn.BatchNorm2d(128)

        # Block 4 Resnet_3
        self.conv4_1 = nn.Conv2d(128, 256, 3, 1, 1) 
        self.bn4_1 = nn.BatchNorm2d(256)
        self.conv4_2 = nn.Conv2d(256, 256, 3, 1, 1)
        self.bn4_2 = nn.BatchNorm2d(256)
        self.conv4_3 = nn.Conv2d(128, 256, 1, 1) 
        self.bn4_3 = nn.BatchNorm2d(256)

        # Block 5 Resnet_4
        self.conv5_1 = nn.Conv2d(256, 512, 3, 1, 1) # 512*32*32
        self.bn5_1 = nn.BatchNorm2d(512)
        self.conv5_2 = nn.Conv2d(512, 512, 3, 1, 1)
        self.bn5_2 = nn.BatchNorm2d(512)  
        self.conv5_3 = nn.Conv2d(256, 512, 1, 1) 
        self.bn5_3 = nn.BatchNorm2d(512) 

        self.fc = nn.Linear(512, 9)
        

    def forward(self, X):

        # Block 1
        X = F.max_pool2d(F.relu(self.bn1(self.conv1(X))), kernel_size=3, stride=2, padding=1) 

        # Block 2 Resnet_1
        Y = F.relu(self.bn2_1(self.conv2_1(X)))
        Y = self.bn2_2(self.conv2_2(Y))
        X = F.relu(Y + X)

        # Block 3 Resnet_2
        Y = F.relu(self.bn3_1(self.conv3_1(X)))
        Y = self.bn3_2(self.conv3_2(Y))
        X = self.bn3_3(self.conv3_3(X))
        X = F.relu(Y + X)

        # Block 4 Resnet_3
        Y = F.relu(self.bn4_1(self.conv4_1(X)))
        Y = self.bn4_2(self.conv4_2(Y))
        X = self.bn4_3(self.conv4_3(X))
        X = F.relu(Y + X)

        # Block 5 Resnet_4
        Y = F.relu(self.bn5_1(self.conv5_1(X)))
        Y = self.bn5_2(self.conv5_2(Y))
        X = self.bn5_3(self.conv5_3(X))
        X = F.relu(Y + X)

        X = F.avg_pool2d(X, kernel_size=X.size()[2:])

        X = X.view(X.size(0), -1)

        output = self.fc(X)
        return F.log_softmax(output)

ResNet_18 = ResNet1()

In [ ]:
## ResNet-34

class ResNet2(nn.Module):
    def __init__(self):

        super(ResNet2, self).__init__()

        # Block 1   # 128*128
        self.conv1 = nn.Conv2d(1, 64, 7, 2, 3) # in_channels, out_channels, kernel_size, stride, padding  # 64*64*64
        self.bn1 = nn.BatchNorm2d(64)

        # Block 2 Resnet_1-1
        self.conv2_1_1 = nn.Conv2d(64, 64, 3, 1, 1)
        self.bn2_1_1 = nn.BatchNorm2d(64)
        self.conv2_2_1 = nn.Conv2d(64, 64, 3, 1, 1)
        self.bn2_2_1 = nn.BatchNorm2d(64)    

        # Block 2 Resnet_1-2
        self.conv2_1_2 = nn.Conv2d(64, 64, 3, 1, 1)
        self.bn2_1_2 = nn.BatchNorm2d(64)
        self.conv2_2_2 = nn.Conv2d(64, 64, 3, 1, 1)
        self.bn2_2_2 = nn.BatchNorm2d(64)   

        # Block 2 Resnet_1-3
        self.conv2_1_3 = nn.Conv2d(64, 64, 3, 1, 1)
        self.bn2_1_3 = nn.BatchNorm2d(64)
        self.conv2_2_3 = nn.Conv2d(64, 64, 3, 1, 1)
        self.bn2_2_3 = nn.BatchNorm2d(64) 

        # Block 3 Resnet_2_1
        self.conv3_1_1 = nn.Conv2d(64, 128, 3, 1, 1)
        self.bn3_1_1 = nn.BatchNorm2d(128)
        self.conv3_2_1 = nn.Conv2d(128, 128, 3, 1, 1)
        self.bn3_2_1 = nn.BatchNorm2d(128)
        self.conv3_3_1 = nn.Conv2d(64, 128, 1, 1)
        self.bn3_3_1 = nn.BatchNorm2d(128)

        # Block 3 Resnet_2_2
        self.conv3_1_2 = nn.Conv2d(128, 128, 3, 1, 1)
        self.bn3_1_2 = nn.BatchNorm2d(128)
        self.conv3_2_2 = nn.Conv2d(128, 128, 3, 1, 1)
        self.bn3_2_2 = nn.BatchNorm2d(128)

        # Block 3 Resnet_2_3
        self.conv3_1_3 = nn.Conv2d(128, 128, 3, 1, 1)
        self.bn3_1_3 = nn.BatchNorm2d(128)
        self.conv3_2_3 = nn.Conv2d(128, 128, 3, 1, 1)
        self.bn3_2_3 = nn.BatchNorm2d(128)

        # Block 3 Resnet_2_4
        self.conv3_1_4 = nn.Conv2d(128, 128, 3, 1, 1)
        self.bn3_1_4 = nn.BatchNorm2d(128)
        self.conv3_2_4 = nn.Conv2d(128, 128, 3, 1, 1)
        self.bn3_2_4 = nn.BatchNorm2d(128)

        # Block 4 Resnet_3_1
        self.conv4_1_1 = nn.Conv2d(128, 256, 3, 1, 1) 
        self.bn4_1_1 = nn.BatchNorm2d(256)
        self.conv4_2_1 = nn.Conv2d(256, 256, 3, 1, 1)
        self.bn4_2_1 = nn.BatchNorm2d(256)
        self.conv4_3_1 = nn.Conv2d(128, 256, 1, 1) 
        self.bn4_3_1 = nn.BatchNorm2d(256)

        # Block 4 Resnet_3_2
        self.conv4_1_2 = nn.Conv2d(256, 256, 3, 1, 1) 
        self.bn4_1_2 = nn.BatchNorm2d(256)
        self.conv4_2_2 = nn.Conv2d(256, 256, 3, 1, 1)
        self.bn4_2_2 = nn.BatchNorm2d(256)

        # Block 4 Resnet_3_3
        self.conv4_1_3 = nn.Conv2d(256, 256, 3, 1, 1) 
        self.bn4_1_3 = nn.BatchNorm2d(256)
        self.conv4_2_3 = nn.Conv2d(256, 256, 3, 1, 1)
        self.bn4_2_3 = nn.BatchNorm2d(256)

        # Block 4 Resnet_3_4
        self.conv4_1_4 = nn.Conv2d(256, 256, 3, 1, 1) 
        self.bn4_1_4 = nn.BatchNorm2d(256)
        self.conv4_2_4 = nn.Conv2d(256, 256, 3, 1, 1)
        self.bn4_2_4 = nn.BatchNorm2d(256)

        # Block 4 Resnet_3_5
        self.conv4_1_5 = nn.Conv2d(256, 256, 3, 1, 1) 
        self.bn4_1_5 = nn.BatchNorm2d(256)
        self.conv4_2_5 = nn.Conv2d(256, 256, 3, 1, 1)
        self.bn4_2_5 = nn.BatchNorm2d(256)

        # Block 4 Resnet_3_6
        self.conv4_1_6 = nn.Conv2d(256, 256, 3, 1, 1) 
        self.bn4_1_6 = nn.BatchNorm2d(256)
        self.conv4_2_6 = nn.Conv2d(256, 256, 3, 1, 1)
        self.bn4_2_6 = nn.BatchNorm2d(256)

        # Block 5 Resnet_4_1
        self.conv5_1_1 = nn.Conv2d(256, 512, 3, 1, 1) # 512*32*32
        self.bn5_1_1 = nn.BatchNorm2d(512)
        self.conv5_2_1 = nn.Conv2d(512, 512, 3, 1, 1)
        self.bn5_2_1 = nn.BatchNorm2d(512)  
        self.conv5_3_1 = nn.Conv2d(256, 512, 1, 1) 
        self.bn5_3_1 = nn.BatchNorm2d(512) 

        # Block 5 Resnet_4_2
        self.conv5_1_2 = nn.Conv2d(512, 512, 3, 1, 1) 
        self.bn5_1_2 = nn.BatchNorm2d(512)
        self.conv5_2_2 = nn.Conv2d(512, 512, 3, 1, 1)
        self.bn5_2_2 = nn.BatchNorm2d(512) 

        # Block 5 Resnet_4_3
        self.conv5_1_3 = nn.Conv2d(512, 512, 3, 1, 1) 
        self.bn5_1_3 = nn.BatchNorm2d(512)
        self.conv5_2_3 = nn.Conv2d(512, 512, 3, 1, 1)
        self.bn5_2_3 = nn.BatchNorm2d(512) 

        self.fc = nn.Linear(512, 9)
        

    def forward(self, X):

        # Block 1
        X = F.max_pool2d(F.relu(self.bn1(self.conv1(X))), kernel_size=3, stride=2, padding=1) 

        # Block 2 Resnet_1_1
        Y = F.relu(self.bn2_1_1(self.conv2_1_1(X)))
        Y = self.bn2_2_1(self.conv2_2_1(Y))
        X = F.relu(Y + X)

        # Block 2 Resnet_1_2
        Y = F.relu(self.bn2_1_2(self.conv2_1_2(X)))
        Y = self.bn2_2_2(self.conv2_2_2(Y))
        X = F.relu(Y + X)

        # Block 2 Resnet_1_3
        Y = F.relu(self.bn2_1_3(self.conv2_1_3(X)))
        Y = self.bn2_2_3(self.conv2_2_3(Y))
        X = F.relu(Y + X)

        # Block 3 Resnet_2_1
        Y = F.relu(self.bn3_1_1(self.conv3_1_1(X)))
        Y = self.bn3_2_1(self.conv3_2_1(Y))
        X = self.bn3_3_1(self.conv3_3_1(X))
        X = F.relu(Y + X)

        # Block 3 Resnet_2_2
        Y = F.relu(self.bn3_1_2(self.conv3_1_2(X)))
        Y = self.bn3_2_2(self.conv3_2_2(Y))
        X = F.relu(Y + X)

        # Block 3 Resnet_2_3
        Y = F.relu(self.bn3_1_3(self.conv3_1_3(X)))
        Y = self.bn3_2_3(self.conv3_2_3(Y))
        X = F.relu(Y + X)

        # Block 3 Resnet_2_4
        Y = F.relu(self.bn3_1_4(self.conv3_1_4(X)))
        Y = self.bn3_2_4(self.conv3_2_4(Y))
        X = F.relu(Y + X)

        # Block 4 Resnet_3_1
        Y = F.relu(self.bn4_1_1(self.conv4_1_1(X)))
        Y = self.bn4_2_1(self.conv4_2_1(Y))
        X = self.bn4_3_1(self.conv4_3_1(X))
        X = F.relu(Y + X)

        # Block 4 Resnet_3_2
        Y = F.relu(self.bn4_1_2(self.conv4_1_2(X)))
        Y = self.bn4_2_2(self.conv4_2_2(Y))
        X = F.relu(Y + X)

        # Block 4 Resnet_3_3
        Y = F.relu(self.bn4_1_3(self.conv4_1_3(X)))
        Y = self.bn4_2_3(self.conv4_2_3(Y))
        X = F.relu(Y + X)

        # Block 4 Resnet_3_4
        Y = F.relu(self.bn4_1_4(self.conv4_1_4(X)))
        Y = self.bn4_2_4(self.conv4_2_4(Y))
        X = F.relu(Y + X)

        # Block 4 Resnet_3_5
        Y = F.relu(self.bn4_1_5(self.conv4_1_5(X)))
        Y = self.bn4_2_5(self.conv4_2_5(Y))
        X = F.relu(Y + X)

        # Block 4 Resnet_3_6
        Y = F.relu(self.bn4_1_6(self.conv4_1_6(X)))
        Y = self.bn4_2_6(self.conv4_2_6(Y))
        X = F.relu(Y + X)

        # Block 5 Resnet_4_1
        Y = F.relu(self.bn5_1_1(self.conv5_1_1(X))) 
        Y = self.bn5_2_1(self.conv5_2_1(Y))
        X = self.bn5_3_1(self.conv5_3_1(X))
        X = F.relu(Y + X)

        # Block 5 Resnet_4_2
        Y = F.relu(self.bn5_1_2(self.conv5_1_2(X)))
        Y = self.bn5_2_2(self.conv5_2_2(Y))
        X = F.relu(Y + X)

        # Block 5 Resnet_4_3
        Y = F.relu(self.bn5_1_3(self.conv5_1_3(X)))
        Y = self.bn5_2_3(self.conv5_2_3(Y))
        X = F.relu(Y + X)

        X = F.avg_pool2d(X, kernel_size=X.size()[2:])


        # This layer is an imaginary one. It simply states that we should see each member of x
        X = X.view(X.size(0), -1)

        output = self.fc(X)
        return F.log_softmax(output)

ResNet_34 = ResNet2()

In [ ]:
def net_and_optm(model, learning_rate):
  if if_use_gpu:
      network = model.cuda()
  else:
      network = model

  optimizer_sgd = optim.SGD(network.parameters(), lr=learning_rate, momentum=0.9, weight_decay=1e-5)

  return network, optimizer_sgd

In [ ]:
train_losses = []
train_counter = []
test_losses = []
test_acc = []

def train(network, epoch, train_loader, test_loader, optimizer_sgd, batchsize):
  network.train()
  correct = 0
  for batch_idx, (data, target) in enumerate(train_loader):
    if if_use_gpu:
      data=data.cuda()
      target=target.cuda()

    optimizer=optimizer_sgd

    optimizer.zero_grad()
    output = network(data)
    if if_use_gpu:
      loss = F.nll_loss(output, target).cuda() # negative log liklhood loss
    else:
      loss = F.nll_loss(output, target)
    loss.backward()
    optimizer.step()
      
    pred = output.data.max(1, keepdim=True)[1]
    correct += pred.eq(target.data.view_as(pred)).sum()

    if batch_idx % 20 == 0:
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        epoch, batch_idx * len(data), len(train_loader.dataset),
        100. * batch_idx / len(train_loader), loss.cpu().item()))
      print("Accuracy: {}/{} ({:.0f}%)".format(correct, len(train_loader.dataset),
         100. * correct / len(train_loader.dataset)))
      train_losses.append(loss.cpu().item())
      train_counter.append(
        (batch_idx * batchsize) + ((epoch)*len(train_loader.dataset)))
      torch.save(network.state_dict(), './model.pth')
      torch.save(optimizer.state_dict(), './optimizer.pth')

def test():
  network.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:
      if if_use_gpu:
        data=data.cuda()
        target=target.cuda()
      output = network(data)
      test_loss += F.nll_loss(output, target, size_average=False).cpu().item()
      pred = output.data.max(1, keepdim=True)[1]
      correct += pred.eq(target.data.view_as(pred)).sum()
  test_loss /= len(test_loader.dataset)
  test_losses.append(test_loss)
  correct_pecent=correct / len(test_loader.dataset)
  test_acc.append(correct_pecent.cpu())
  print('\nTest set: Avg. loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
      test_loss, correct, len(test_loader.dataset),
      100. * correct_pecent))

In [ ]:
def train_epochs(network, num_epoch, train_loader, test_loader, optimizer_sgd, batchsize):
  for epoch in range(num_epoch):
    start = time.time()
    train(network, epoch, train_loader, test_loader, optimizer_sgd, batchsize)
    duration = time.time() - start 
    print('Training duation: %.4f'%duration)
    test(network, test_loader)

  # Training loss
  fig = plt.figure() 
  plt.plot(train_counter, train_losses, color='blue')
  plt.legend(['Train Loss'], loc='upper right')
  plt.xlabel('number of training examples seen')
  plt.ylabel('negative log likelihood loss')
  import seaborn as sns
  import pandas as pd
  sns.set()
  sns.lineplot(data=pd.DataFrame(test_acc))

In [ ]:
class TestDataset(Dataset):
    def __init__(self, img_file, transform=None, idx = None):
        self.data = pickle.load( open( img_file, 'rb' ), encoding='bytes')
        if idx is not None:
          self.data = self.data[idx]
        self.transform = transform

    def __len__(self):
        return 10000

    def __getitem__(self, index):
        img = self.data[index] 
        img = Image.fromarray(img.astype('uint8'), mode='L')
        
        if self.transform is not None:
          img = self.transform(img)

        return img

In [ ]:
# Prediction for test dataset
def prediction(batchsize, network):
  testdata = TestDataset('./Test.pkl',transform=img_transform, idx=None)
  def _init_fn(worker_id):
    np.random.seed(int(seed)+worker_id)
  testdata_loader = DataLoader(testdata, batch_size=batchsize, shuffle=False, num_workers=16, pin_memory=True, worker_init_fn=_init_fn)  

  prediction = [1]
  for data in testdata_loader:
      if if_use_gpu:
        data=data.cuda()
      output = network(data)
      pred = output.data.max(1, keepdim=True)[1].cpu().numpy()
      prediction = np.vstack([prediction, pred])

  Test_pred = prediction[1:,:] + 5

  return Test_pred

In [ ]:
import pandas as pd
def main(model, batchsize, learning_rate, num_epoch):
  train_loader, test_loader = data_loader(batchsize)
  network, optimizer_sgd = net_and_optm(model, learning_rate)
  train_epochs(network, num_epoch, train_loader, test_loader, optimizer_sgd, batchsize)
  Test_pred = prediction(batchsize, network)
  # Save to CSV
  save = pd.DataFrame(Test_pred) 
  save.to_csv("Test_pred.csv", header = ['class'], index=True)

In [ ]:
main(model = net, batchsize = 64, learning_rate = 0.05, num_epoch = 40)